In [1]:
import glob

In [2]:
list_files = glob.glob("/HFIR/CG1D/IPTS-33592/raw/ct_scans/2024_07_03_lam_circuit_small/*.tiff")


# using tifffile 

In [8]:
import tifffile
import tqdm

In [4]:
# data = []
# for _file in list_files:
#     data.append(tifffile.imread(_file))

In [5]:
print("Done!")

Done!


# using dxchange

In [6]:
import dxchange
import re
import numpy as np
from pathlib import Path

/SNS/users/j35/mambaforge/envs/tomoornl310/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [7]:
data = np.zeros((len(list_files), 6400, 6400))
for _index, _file in enumerate(list_files):
    data[_index] = dxchange.read_tiff(_file)

data = [_data for _data in data if data is not None]

In [4]:
def _extract_rotation_angles(filelist, metadata_idx: int = 65039):

    regex = r"\d{8}_\S*_\d{4}_(?P<deg>\d{3})_(?P<dec>\d{3})_\d*\.tiff"
    matches = [re.match(regex, Path(f).name) for f in filelist]
    if all(matches):
        logger.info("Using rotation angles from filenames.")
        rotation_angles = np.array([float(".".join(m.groups())) for m in matches])
    else:
        # extract rotation angles from metadata
        file_ext = set([Path(f).suffix for f in filelist])
        if file_ext != {".tiff"}:
            logger.error("Only tiff files are supported.")
            raise ValueError("Rotation angle from metadata is only supported for Tiff.")
        # -- read metadata
        # img = tifffile.TiffFile("test_with_metadata_0.tiff")
        # img.pages[0].tags[65039].value
        # >> 'RotationActual:0.579840'
        rotation_angles = np.array(
            [float(tifffile.TiffFile(f).pages[0].tags[metadata_idx].value.split(":")[-1]) for f in filelist],
            dtype="float",
        )
    return rotation_angles

In [10]:
_extract_rotation_angles(list_files)

array([216.92, 188.87, 341.7 , ..., 325.55, 321.13,  48.79])